In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import datetime
#turns seconds into a string format ffmpeg uses
def seconds_formatter(sec):
    x = "0" + str(datetime.timedelta(seconds=sec))[:-3]
    return x

In [5]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\A_copiedDeepCut_resnet50_goprolensfrontJun25shuffle1_60002.h5")
#remove extra level "scorer"
df.columns = df.columns.droplevel()
df.describe()

bodyparts          Nose                                    Pellet  \
coords                x             y    likelihood             x   
count      55026.000000  55026.000000  55026.000000  55026.000000   
mean        1330.184539    646.782603      0.376942   1379.689905   
std          201.531635    222.608088      0.389379    420.195539   
min            0.370821     -5.005973      0.000485      0.487646   
25%         1317.909296    564.589215      0.020343   1341.302102   
50%         1340.824574    687.884438      0.172433   1371.627043   
75%         1383.259692    789.351130      0.815848   1420.980582   
max         2524.008662   1520.601292      0.998947   2688.581522   

bodyparts                                 DomInside                \
coords                y    likelihood             x             y   
count      55026.000000  55026.000000  55026.000000  55026.000000   
mean         890.630344      0.353468   1348.746314    698.338131   
std          352.130231      0.381447    177.198718    190.104141   
min            0.977137      0.000128     -1.745432     -2.163946   
25%          679.328163      0.030228   1315.735032    609.551925   
50%          934.700097      0.138022   1355.566452    714.161687   
75%         1034.587177      0.825975   1411.890644    801.298333   
max         1525.692149      0.997842   2539.297546   1519.157574   

bodyparts                  DomOutside      ...             Middle  \
coords       likelihood             x      ...         likelihood   
count      55026.000000  55026.000000      ...       55026.000000   
mean           0.042437   1345.995294      ...           0.073249   
std            0.148373    184.583817      ...           0.170021   
min            0.000050     -3.968345      ...           0.000058   
25%            0.002453   1302.127836      ...           0.003987   
50%            0.005473   1339.641322      ...           0.011672   
75%            0.014076   1411.649381      ...           0.041728   
max            0.997374   2542.607008      ...           0.987304   

bodyparts          Ring                                     Pinky  \
coords                x             y    likelihood             x   
count      55026.000000  55026.000000  55026.000000  55026.000000   
mean        1347.171692    721.797350      0.066174   1346.244365   
std          204.993579    178.393742      0.158645    203.967791   
min           -4.533683     -1.481583      0.000048     -3.420242   
25%         1308.492969    652.538706      0.004266   1304.805376   
50%         1339.759142    721.430287      0.013019   1339.186867   
75%         1405.858760    814.640043      0.040963   1413.889349   
max         2701.087938   1517.565257      0.985866   2704.617106   

bodyparts                                 OtherHand                \
coords                y    likelihood             x             y   
count      55026.000000  55026.000000  55026.000000  55026.000000   
mean         726.047859      0.040276   1410.490640    692.802901   
std          180.706104      0.133038    157.962020    158.586212   
min           -2.252955      0.000062      1.162153     -1.754390   
25%          652.930016      0.002084   1357.557903    617.124485   
50%          728.146373      0.005336   1424.446016    677.042769   
75%          811.043618      0.015188   1465.638888    773.248826   
max         1518.772220      0.986586   2538.984463   1519.771770   

bodyparts                
coords       likelihood  
count      55026.000000  
mean           0.177920  
std            0.261808  
min            0.000121  
25%            0.012334  
50%            0.053886  
75%            0.212803  
max            0.997119  

[8 rows x 27 columns]

In [6]:
def frame_to_time(frame):
    total_seconds = (frame/55026)*459
    return round(total_seconds, 3)
def time_to_frame(time):
    return int(round(55026*(time/459)))
def dist_from_dispenser(x, y):
    return ((x-1368)**2 + (y-935)**2)**0.5
def outside_box(x, y):
    return y > 300

In [7]:
labels = {key: df[df[key, 'likelihood'] > 0.9][key] for key in list(df.columns.levels[0])}
{labels[key].insert(0, "frame", labels[key].index, True) for key in labels.keys()}
{labels[key].insert(0, "time", labels[key]['frame'].map(frame_to_time), True) for key in labels.keys()}
nose = labels['Nose']
otherhand = labels['OtherHand']
dominside = labels['DomInside']
domoutside = labels['DomOutside']
index = labels['Index']
middle = labels['Middle']
ring = labels['Ring']
pinky = labels['Pinky']
pellet = labels['Pellet']
# pellet[(pellet['time'] > 39) & (pellet['time'] < 40)].plot(kind='scatter', x='x', y='y')
# pellet.plot(kind='scatter', x='x', y='y')
# nose.plot(kind='scatter', x='x', y='y')

## Random Forest Classifier
identify reaches, successes, failures

In [30]:
true_reach_times = [1.153, 1.844, 14.02, 14.538, 15.724, 16.227, 18.416, 21.184, 27.747, 32.237, 33.952, 37.731, 
                38.378, 41.412, 42.066, 42.364, 45.69, 48.501, 50.928, 55.111, 56.685, 61.739, 62.336, 
                62.747, 64.524, 66.032, 76.965, 83.258, 90.252, 105.25, 105.67, 109.465, 113.356, 113.994, 121.798,
                124.059, 132.204, 132.597, 135.263, 135.633, 136.588, 138.327, 145.304, 154.300, 170.547, 
                182.046, 190.137, 202.976, 240.157, 243.168, 253.75, 258.548, 272.461, 281.791, 284.774, 
                295.074, 303.63, 310.3, 323.374, 351.034, 364.535, 376.356, 432.654]
true_reach_frames = [time_to_frame(x) for x in true_reach_times]
true_reach_labels = ['r', 'r', 'f', 'f', 'r', 'r', 'r', 's', 'r', 'r', 's', 'r', 'r', 'r', 'f', 'r', 'f', 'r', 's',
                     'r', 's', 'r', 'r', 'r', 'r', 's', 's', 's', 's', 'f', 'r', 'r', 'f', 'f', 'r', 's', 'f', 'r',
                     'r', 'r', 'r', 's', 's', 's', 's', 'f', 's', 's', 'r', 's', 'f', 'f', 's', 'r', 's', 's', 'r',
                     's', 'f', 'f', 'f', 's', 's']

In [42]:
import random
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
def create_classifier(features, labels, n_trees, test_proportion=0.25, random_seed=76, dictionary={}):
    train_feats, test_feats, train_labs, test_labs = train_test_split(features, labels, test_size = test_proportion, random_state = random_seed)
    print('Training Features Shape:', train_feats.shape)
    print('Training Labels Shape:', train_labs.shape)
    print('Testing Features Shape:', test_feats.shape)
    print('Testing Labels Shape:', test_labs.shape)
    rf = RandomForestClassifier(n_estimators = n_trees, random_state = random_seed)
    # Train the model on training data
    rf.fit(train_feats, train_labs)
    # Use the forest's predict method on the test data
    predictions = rf.predict(test_feats)
    print(predictions)
    print(test_labs)
    wrong_labels = []
    for i in range(len(predictions)):
        if predictions[i] != test_labs[i]:
            frame = -1
            for key in dictionary.keys():
                if (dictionary[key] == test_feats[i]).all():
                    frame = key
                    break
            wrong_labels.append((frame_to_time(frame), predictions[i], test_labs[i]))
            predictions[i] = 'X'
    print('Wrong Labels', wrong_labels)
    return rf, wrong_labels
def get_dataset(df, cols, start_frames, frames_before, frames_after):
    dataframe = df[cols]
    dataset = []
    for start_f in start_frames:
        temp_df = dataframe.iloc[start_f - frames_before: start_f + frames_after]
        dataset.append(temp_df.values.flatten())
    return np.asarray(dataset)
def get_predicted_frames(classifier, dataframe, columns, frames_before, frames_after, step_size):
    dataframe = dataframe[columns]
    inputs = []
    num_frames = frames_before + frames_after
    for index in range(num_frames, dataframe.shape[0] - num_frames, step_size):
        temp_df = dataframe.iloc[index - frames_before: index + frames_after]
        inputs.append(temp_df.values.flatten())
    inputs = np.asarray(inputs)
    predictions = classifier.predict(inputs)
    return predictions
def get_non_reach_frames(num_frames, blacklist, avoidance_radius, random_seed = 75):
    non_reaches = []
    random.seed(random_seed)
    while len(non_reaches) < num_frames:
        #hard code 3 non-reaches
        frame = random.randint(10, 24000)

        too_close = False
        for start_time in blacklist:
            if abs(frame - start_time) < avoidance_radius:
                print('no')
                too_close = True
                break
        if not too_close:
            non_reaches.append(frame)
    return non_reaches

In [98]:
true_reach_times = [1.153, 1.844, 14.02, 14.538, 15.724, 16.227, 18.416, 21.184, 27.747, 32.237, 33.952, 37.731, 
                38.378, 41.412, 42.066, 42.364, 45.69, 48.501, 50.928, 55.111, 56.685, 61.739, 62.336, 
                62.747, 64.524, 66.032, 76.965, 83.258, 90.252, 105.25, 105.67, 109.465, 113.356, 113.994, 121.798,
                124.059, 132.204, 132.597, 135.263, 135.633, 136.588, 138.327, 145.304, 154.300, 170.547, 
                182.046, 190.137, 202.976, 240.157, 243.168, 253.75, 258.548, 272.461, 281.791, 284.774, 
                295.074, 303.63, 310.3, 323.374, 351.034, 364.535, 376.356, 432.654]
true_reach_frames = [time_to_frame(x) for x in true_reach_times]
true_reach_labels = ['r', 'r', 'f', 'f', 'r', 'r', 'r', 's', 'r', 'r', 's', 'r', 'r', 'r', 'f', 'r', 'f', 'r', 's',
                     'r', 's', 'r', 'r', 'r', 'r', 's', 's', 's', 's', 'f', 'r', 'r', 'f', 'f', 'r', 's', 'f', 'r',
                     'r', 'r', 'r', 's', 's', 's', 's', 'f', 's', 's', 'r', 's', 'f', 'f', 's', 'r', 's', 's', 'r',
                     's', 'f', 'f', 'f', 's', 's']
non_reaches_frames = get_non_reach_frames(70, true_reach_frames, 12)
true_reach_labels[true_reach_times.index(83.258)] = 'f'
true_reach_labels[true_reach_times.index(90.252)] = 'f'

In [102]:
features = get_dataset(df, ['Nose', 'DomInside', 'Pellet', 'Index'], true_reach_frames+non_reaches_frames, 24, 60)
dictionary = dict(zip(true_reach_frames+non_reaches_frames, features))
labels = np.asarray(true_reach_labels + ['n' for x in non_reaches_frames])

In [115]:
blue = create_classifier(features, labels, 1000, random_seed=3, dictionary=dictionary, test_proportion=0.01)

Training Features Shape: (131, 1008)
Training Labels Shape: (131,)
Testing Features Shape: (2, 1008)
Testing Labels Shape: (2,)
['s' 'n']
['s' 'n']
Wrong Labels []


In [104]:
wrong_ones = []
for i in range(100, 200):
    wrong_ones.extend(create_classifier(features, labels, 1000, random_seed=i, dictionary=dictionary, test_proportion=0.1)[1])

Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['f' 'n' 's' 'n' 'f' 'n' 's' 'n' 's' 'r' 'n' 'f' 'f' 'n']
['f' 'n' 's' 'n' 'f' 'n' 's' 'n' 's' 'r' 'n' 'f' 'f' 'n']
Wrong Labels []
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['s' 'n' 'n' 'f' 'n' 'r' 'n' 'n' 'n' 'n' 's' 'r' 'r' 'r']
['s' 'n' 'n' 'f' 'n' 'r' 'n' 'n' 'n' 'n' 's' 'r' 'r' 'r']
Wrong Labels []
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['f' 's' 'n' 'r' 'r' 'n' 's' 'n' 'n' 'n' 'f' 'n' 'n' 's']
['f' 's' 'n' 'r' 'r' 'n' 's' 'n' 'n' 'n' 'f' 'n' 'n' 's']
Wrong Labels []
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['n' 'n' 'n' 'n' 'f' 's' 'n' 'f' 'n' 'f' 'n' 'n' 's' 'n']
['n' 'n' 'n' 'n' 'f' 's' '

['n' 's' 's' 'n' 'n' 'n' 's' 'f' 'n' 'r' 'n' 'n' 'n' 'r']
['n' 's' 's' 'n' 'n' 'n' 'n' 'f' 'n' 'r' 'n' 'n' 'n' 'r']
Wrong Labels [(135.8, 's', 'n')]
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['r' 'n' 'n' 'n' 'r' 'n' 's' 'r' 'n' 'n' 'f' 'n' 'n' 's']
['r' 'n' 'n' 'n' 'r' 'n' 's' 'r' 'n' 'n' 'f' 'n' 'n' 's']
Wrong Labels []
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['f' 'n' 'f' 's' 'n' 'n' 'n' 'n' 'n' 's' 'n' 'n' 'n' 'r']
['f' 'n' 'f' 's' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'r']
Wrong Labels [(135.8, 's', 'n')]
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['s' 'f' 'r' 'r' 'f' 'r' 'r' 'r' 's' 'n' 'r' 'r' 's' 'n']
['s' 'f' 'r' 'r' 'r' 'r' 'r' 'r' 's' 'n' 'r' 'r' 's' 'n']
Wrong Labels [(240.16, 'f', 'r')]
Training Features Shape: (119,

['r' 's' 'n' 'n' 'n' 'r' 'n' 's' 'f' 'n' 's' 'n' 'n' 'n']
['r' 's' 'n' 'n' 'n' 'r' 'n' 's' 'f' 'n' 's' 'n' 'n' 'n']
Wrong Labels []
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['s' 'r' 'f' 'n' 'n' 'r' 's' 'n' 'n' 's' 'n' 'n' 'r' 'n']
['s' 'r' 'f' 'n' 'n' 'r' 's' 'n' 'n' 's' 'n' 'n' 'r' 'n']
Wrong Labels []
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['r' 's' 'n' 'n' 'n' 'f' 'r' 's' 'n' 'f' 'n' 's' 'n' 'n']
['r' 'n' 'n' 'n' 'n' 'f' 'r' 's' 'n' 'f' 'n' 's' 'n' 'n']
Wrong Labels [(135.8, 's', 'n')]
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['s' 'r' 'r' 'n' 'n' 'n' 'f' 'n' 'f' 's' 's' 's' 'r' 'n']
['s' 'r' 'r' 'n' 'n' 'n' 'f' 'n' 'f' 's' 's' 's' 'r' 'n']
Wrong Labels []
Training Features Shape: (119, 1008)
Training Labels Shape: (119,

['f' 'n' 's' 's' 'r' 'r' 'r' 'n' 'n' 'r' 'n' 'r' 'r' 'r']
['f' 'n' 's' 's' 'r' 'r' 'r' 'n' 'n' 'r' 'n' 'r' 'r' 'r']
Wrong Labels []
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['n' 'r' 'n' 'r' 'r' 'r' 'n' 'f' 'n' 'n' 's' 'n' 'n' 'n']
['n' 'r' 'n' 'r' 'r' 'r' 'n' 'f' 'n' 'n' 's' 'n' 'n' 'n']
Wrong Labels []
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['s' 'n' 'n' 'n' 's' 'r' 'n' 's' 'n' 's' 'n' 'n' 's' 'n']
['s' 'n' 'n' 'n' 's' 'r' 'n' 's' 'n' 's' 'n' 'n' 's' 'n']
Wrong Labels []
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Features Shape: (14, 1008)
Testing Labels Shape: (14,)
['f' 'n' 'f' 's' 'r' 'n' 'n' 'n' 'f' 'r' 'n' 'n' 's' 'n']
['f' 'n' 'f' 's' 'r' 'n' 'n' 'n' 'f' 'r' 'n' 'n' 's' 'n']
Wrong Labels []
Training Features Shape: (119, 1008)
Training Labels Shape: (119,)
Testing Feature

In [105]:
wrong_ones.sort(key= lambda x: x[0])
wrong_ones

[(14.539, 'r', 'f'),
 (14.539, 'r', 'f'),
 (14.539, 'r', 'f'),
 (14.539, 'r', 'f'),
 (76.967, 'f', 's'),
 (135.266, 'f', 'r'),
 (135.8, 's', 'n'),
 (135.8, 's', 'n'),
 (135.8, 's', 'n'),
 (135.8, 's', 'n'),
 (135.8, 's', 'n'),
 (135.8, 's', 'n'),
 (135.8, 's', 'n'),
 (135.8, 's', 'n'),
 (135.8, 's', 'n'),
 (135.8, 's', 'n'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r'),
 (240.16, 'f', 'r')]

In [106]:
wrong_df = pd.DataFrame(wrong_ones, columns = ['time', 'guess', 'actual'])
wrong_df

,time,guess,actual
0,14.539,r,f
1,14.539,r,f
2,14.539,r,f
3,14.539,r,f
4,76.967,f,s
5,135.266,f,r
6,135.800,s,n
7,135.800,s,n
8,135.800,s,n
9,135.800,s,n


In [108]:
wrong_df.groupby(wrong_df.columns.tolist(),as_index=False).size()

time     guess  actual
14.539   r      f          4
76.967   f      s          1
135.266  f      r          1
135.800  s      n         10
240.160  f      r         15
dtype: int64

In [109]:
# 182: pellet goes up like a success?
# 240: pellet dispenser goes down during reach
# 83, 90: mouse drops pellet within/near acrylic, ambigious even to humans
# 

In [138]:
def get_predicted_times(classifier, dataframe, columns, frames_before, frames_after, step_size):
    dataframe = dataframe[columns]
    inputs = []
    num_frames = frames_before + frames_after
    for index in range(num_frames, dataframe.shape[0] - num_frames, step_size):
        temp_df = dataframe.iloc[index - frames_before: index + frames_after]
        inputs.append(temp_df.values.flatten())
    inputs = np.asarray(inputs)
    predictions = classifier.predict(inputs)
    return predictions
start_frames = range(15, df2.shape[0] - 15, 6)
predictions = get_predicted_times(blue[0], df2, ['Nose', 'DomInside', 'Pellet', 'Index'], 24, 60, 3)


In [139]:
predicted_reaches = []
predicted_fails = []
predicted_successes = []
for start, prediction in zip(start_frames, predictions):
    if prediction == 'r':
        predicted_reaches.append(frame_to_time(start))
    elif prediction == 'f':
        predicted_fails.append(frame_to_time(start))
    elif prediction == 's':
        predicted_successes.append(frame_to_time(start))
print((predicted_reaches))
predictions

[9.485, 9.535, 9.585, 9.635, 9.684, 9.734, 10.382, 10.431, 10.481, 10.531, 10.581, 10.63, 10.68, 10.73, 11.577, 11.626, 21.535, 21.585, 21.634, 21.684, 21.734, 21.784, 25.07, 25.12, 26.016, 26.066, 26.116, 26.165, 26.215, 26.265, 27.111, 27.161, 27.211, 27.261, 27.311, 27.36, 27.41, 27.46, 27.51, 31.045, 31.095, 31.145, 31.194, 32.34, 33.634, 33.684, 33.734, 33.784, 33.833, 33.883, 33.933, 33.983, 38.016, 38.066, 38.115, 38.165, 38.215, 38.265, 38.315, 38.364, 38.414, 39.31, 39.36, 39.41, 39.46, 39.51, 39.559, 39.609, 39.659, 39.709, 39.758, 39.808, 39.858, 42.447, 42.497, 42.547, 42.597, 42.646, 42.696, 42.746, 42.796, 42.846, 42.895, 42.945, 42.995, 43.045, 46.48, 46.53, 46.58, 46.679, 52.804, 52.854, 52.903, 52.953, 53.003, 53.849, 61.667, 61.717, 61.766, 61.816, 61.866, 62.613, 62.663, 62.712, 62.762, 62.812, 62.862, 62.912, 62.961, 63.011, 63.061, 63.111, 63.16, 63.21, 63.26, 63.31, 64.754, 64.804, 64.853, 64.903, 64.953, 65.003, 65.053, 66.048, 66.098, 66.148, 66.198, 66.248, 66.

array(['n', 'n', 'n', ..., 'n', 'n', 'n'], dtype='<U1')

In [140]:
def get_starts(lst):
    starts = [lst[0]]
    for i in range(1, len(lst)):
        if lst[i] - lst[i-1] > 0.2:
            starts.append(lst[i])
    return starts

In [141]:
predicted_fails = get_starts(predicted_fails)
predicted_reaches = get_starts(predicted_reaches)
predicted_successes = get_starts(predicted_successes)
print(predicted_fails)
print(predicted_reaches)
print(predicted_successes)

[25.17, 26.315, 31.244, 32.091, 53.053]
[9.485, 10.382, 11.577, 21.535, 25.07, 26.016, 27.111, 31.045, 32.34, 33.634, 38.016, 39.31, 42.447, 46.48, 52.804, 53.849, 61.667, 62.613, 64.754, 66.048, 69.882]
[11.676, 46.63, 53.899]


In [136]:
predicted_fails = get_starts(predicted_fails)
predicted_reaches = get_starts(predicted_reaches)
predicted_successes = get_starts(predicted_successes)
print(predicted_fails)
print(predicted_reaches)
print(predicted_successes)

[12.672, 13.22, 15.709, 16.108, 26.614]
[4.805, 5.253, 5.85, 10.83, 12.622, 13.07, 13.618, 15.61, 16.904, 19.095, 19.742, 21.286, 23.327, 26.464, 30.896, 31.394, 32.439, 33.086, 35.028, 50.414, 50.912, 54.248, 58.729, 59.227, 60.322, 61.567, 62.912, 65.202, 66.248, 66.795, 71.725, 72.322, 74.613]
[5.9, 23.377, 27.012, 45.186, 65.501]


In [132]:
df2 = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\6_26_2.7k_copyDeepCut_resnet50_goprolensfrontJun25shuffle1_60005.h5")
df2.columns = df2.columns.droplevel()
df2.describe()

bodyparts         Nose                                 Pellet               \
coords               x            y   likelihood            x            y   
count      9351.000000  9351.000000  9351.000000  9351.000000  9351.000000   
mean       1296.068329   712.373100     0.596747  1593.574426   755.882076   
std         268.476174   184.165988     0.420628   599.865952   298.616968   
min         254.734869    -1.454122     0.000350    80.509280     5.210910   
25%        1327.784033   615.399295     0.104010  1366.093157   439.438768   
50%        1349.254684   755.527213     0.831531  1417.818774   831.444801   
75%        1366.450449   854.077407     0.995138  2227.220030  1013.546118   
max        2701.914486  1240.004734     0.999876  2515.290060  1519.577064   

bodyparts                 DomInside                             DomOutside  \
coords      likelihood            x            y   likelihood            x   
count      9351.000000  9351.000000  9351.000000  9351.000000  9351.000000   
mean          0.284088  1399.162157   740.662430     0.147688  1383.185492   
std           0.339678   261.066181   151.956345     0.308794   272.220221   
min           0.000681    58.679360     1.231414     0.000099    60.762307   
25%           0.023219  1343.926980   639.297090     0.004451  1324.152745   
50%           0.099606  1382.818230   766.973946     0.012470  1353.332562   
75%           0.477487  1469.086784   850.070271     0.050094  1437.360596   
max           0.999795  2701.110523  1404.526586     0.999816  2698.696588   

bodyparts     ...            Middle         Ring                            \
coords        ...        likelihood            x            y   likelihood   
count         ...       9351.000000  9351.000000  9351.000000  9351.000000   
mean          ...          0.178330  1331.743050   795.869920     0.172957   
std           ...          0.317850   405.258878   212.424541     0.305141   
min           ...          0.000085   247.119315     1.010919     0.000164   
25%           ...          0.005416  1299.982876   665.548371     0.006581   
50%           ...          0.013812  1350.837383   828.905748     0.020901   
75%           ...          0.128862  1424.264249   912.532716     0.125315   
max           ...          0.998116  2701.673587  1403.244175     0.998869   

bodyparts        Pinky                              OtherHand               \
coords               x            y   likelihood            x            y   
count      9351.000000  9351.000000  9351.000000  9351.000000  9351.000000   
mean       1381.412165   764.828215     0.113977  1466.566703   712.681945   
std         307.147716   168.987851     0.256527   152.829811   109.310601   
min         141.844064    -1.543287     0.000104    59.115824   162.617403   
25%        1321.890818   657.732046     0.003668  1430.704622   628.015717   
50%        1353.970001   797.315087     0.010950  1459.868799   737.841134   
75%        1445.420568   876.351606     0.027442  1529.373132   800.702176   
max        2698.795418  1451.070842     0.996238  2512.194607  1383.916615   

bodyparts               
coords      likelihood  
count      9351.000000  
mean          0.454486  
std           0.388503  
min           0.000138  
25%           0.035938  
50%           0.409248  
75%           0.878595  
max           0.998284  

[8 rows x 27 columns]

In [126]:
def get_classified_starts(classifier, dataframe, cols, classes, frames_before, frames_after, step_size)
    predictions2 = get_predicted_times(blue[0], df2, ['Nose', 'DomInside', 'Pellet', 'Index'], 24, 120, 6)
    class_to_predictions = {}
    for start, prediction in zip(start_frames, predictions2):
        for c in classes:
            if prediction == c:
                if c in class_to_predictions.keys():
                    class_to_predictions[c].append(start)
                class_to_predictions[c] = 
        if prediction == 'r':
            predicted_reaches.append(frame_to_time(start))
        elif prediction == 'f':
            predicted_fails.append(frame_to_time(start))
        elif prediction == 's':
            predicted_successes.append(frame_to_time(start))
    predicted_fails = get_starts(predicted_fails)
    predicted_reaches = get_starts(predicted_reaches)
    predicted_successes = get_starts(predicted_successes)
print(predicted_fails)
print(predicted_reaches)
print(predicted_successes)

SyntaxError: invalid syntax (<ipython-input-126-778ff6eff707>, line 1)

In [133]:
def frame_to_time(frame):
    total_seconds = (frame/9351)*77.6
    return round(total_seconds, 3)
def time_to_frame(time):
    return int(round(9351*(time/77.6)))

In [137]:
frame_to_time(6)

0.05